Клевцов Тимур, М05-013д

In [ ]:
import math
import plotly
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import networkx as nx
import random
import scipy as sc
from sklearn import datasets
from itertools import *

In [ ]:
N = 200
E = 3     #число ближайших соседей
radius = 1.0  #радиус поиска, используется сейчас
a = 1.0   #ускорение спуска
L = 0.01  #зашумленность
#X, color = datasets.make_swiss_roll(N)
#может работать до 30с, но это того стоит
X, color = datasets.make_s_curve(N)
number = [str(i) for i in range(N)]
fig = go.Figure(data=[go.Scatter3d(
 x=X[:,0],  
 y=X[:,1],
 z=X[:,2],
 mode='markers',
 marker=dict(size=5, color=color, colorscale='Viridis', opacity=0.8),
 text = number
)])
fig.update_layout(  
    autosize=False,
    width=700,
    height=700,
    margin=dict(l=50, r=50, b=50, t=50, pad=4)
)

G = nx.Graph()
D = np.array([[0.0] * N for i in range(N)])
Y = np.array([[0.0] * N for i in range(N)])
y = np.array([[0.0] * 2 for i in range(N)])
gradient = np.array([[0.0] * 2 for i in range(N)])
r_vector = np.array([[0.0] * 2 for i in range(N)])
support = np.array([0.0] * N)
lenght = 0

def len_e(x,y):
  if len(x) != len(y):
    return -1
  dim = len(x)
  return math.sqrt(sum([(x[i] - y[i]) ** 2 for i in range(dim)]))

def create_graph():
  infty = 100000.0
  for i in range(N):
    G.add_node(i)
  for i in range(N):
    for j in range(i + 1, N):

      if i != j:
        l = len_e(X[i],X[j])
        if l < radius:
          exist_n = True
          G.add_edge(i, j, weight=l)
          G.add_edge(j, i, weight=l)
      """
      if i != j:
        support[j] = len_e(X[i], X[j])
    support[i] = infty
    for j in range(E):
      shortest = support.argmin()
      G.add_edge(i, shortest, weight=support[shortest])
      support[shortest] = infty + 1
      """
  return

def create_D():
  for i in lenght:
    for j in range(N):
      if j in i[1]:
        D[i[0]][j] = i[1][j]
      else:
        print(i[0], j, i[1])
  return

def create_start_y():
  L = D.max() / 2
  random.seed()
  for i in range(N):
    y[i][0] = random.uniform(0.0, L)
    y[i][1] = random.uniform(0.0, L)
  return

def count_Y():
  for i in range(N):
    for j in range(i, N):
      l = len_e(y[i],y[j])
      Y[i][j] = l
      Y[j][i] = l
  return

def Function():
  count_Y()
  U = 0.0
  for i in range(N):
    for j in range(i):
      U += (D[i][j] - Y[i][j]) ** 2
  return math.sqrt(U)

def F(x):
  U = 0.0
  for i in range(N):
    for j in range(i):
      dx = math.sqrt((x[2 * i] - x[2 * j]) ** 2 + (x[2 * i + 1] - x[2 * j + 1]) ** 2)
      U += (D[i][j] -dx) ** 2
  return math.sqrt(U)

def grad_F(x):
  grad = np.array([0.0] * 2 * N)
  f = F(x)
  for k in range(N):
    summ_0 = 0.0
    summ_1 = 0.0
    for i in range(N):
      if i != k:
        Y_ik = math.sqrt((x[2 * i] - x[2 * k]) ** 2 + (x[2 * i + 1] - x[2 * k + 1]) ** 2)
        summ_0 += (D[i][k] - Y_ik) * (x[2 * k] - x[2 * i]) / Y_ik
        summ_1 += (D[i][k] - Y_ik) * (x[2 * k + 1] - x[2 * i + 1]) / Y_ik
    grad[2 * k] = - summ_0 / f
    grad[2 * k + 1] = -summ_1 / f
  return grad

def gradient_F(F):
  for k in range(N):
    summ = 0.0
    for i in range(N):
      if i != k:
        summ += (D[i][k] - Y[i][k]) * (y[k][0] - y[i][0]) / Y[i][k]
    gradient[i][0] = - summ / F
    summ = 0.0
    for i in range(N):
      if i != k:
        summ += (D[i][k] - Y[i][k]) * (y[k][1] - y[i][1]) / Y[i][k]
    gradient[i][1] = -summ / F
  summ = 0.0
  for i in range(N):
    summ += gradient[i][0] ** 2 + gradient[i][1] ** 2
  return math.sqrt(summ)

def minimum():
  create_start_y()
  x0 = np.array([0.0] * 2 * N)
  for i in range(2 * N):
    x0[i] = y[i // 2][i % 2] 

  result = sc.optimize.minimize(F, x0, method='BFGS', jac=grad_F,
    options={'gtol':0.1, 'disp': False})
  
  print("отклонение:", result.fun)
  for i in range(2 * N):
    y[i // 2][i % 2] = result.x[i]
  return

def generate_random_vector():
  random.seed()
  for i in range(N):
    r_vector[i][0] = random.uniform(-1.0 * L, 1.0 * L)
    r_vector[i][1] = random.uniform(-1.0 * L, 1.0 * L)
  return


create_graph()
lenght = nx.all_pairs_dijkstra_path_length(G, weight='weight')
create_D()
d = 0
U = 10000.0
while d < 0:
  create_start_y()
  iter = 0
  modul = 1000.0
  while modul > 0.1 and iter < 100:
    U = Function()
    modul = gradient_F(U)
    generate_random_vector()
    y -= (a * gradient +  L * r_vector)
    if iter % 100 == 0:
      print(d, U, modul)
      #print(r_vector[:10])
    iter += 1
  print(U)
  d += 1

minimum()

отклонение: 4.470760450372576


In [ ]:
fig_rezult = go.Figure(data=[go.Scatter(
 x=y[:,0],  
 y=y[:,1],
 mode='markers',
 marker=dict(size=10, color=color, colorscale='Viridis', opacity=0.8),
)])
fig_rezult.update_layout(  
    autosize=False,
    width=700,
    height=700,
    margin=dict(l=50, r=50, b=50, t=50, pad=4)
)
fig_rezult.show()
fig.show()